In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [93]:
df = pd.read_csv("../input/raw_train_regression.csv")

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    284780 non-null  int64  
 1   StockCode    284780 non-null  int64  
 2   Description  284780 non-null  int64  
 3   Quantity     284780 non-null  int64  
 4   InvoiceDate  284780 non-null  object 
 5   UnitPrice    284780 non-null  float64
 6   CustomerID   284780 non-null  float64
 7   Country      284780 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 17.4+ MB


In [95]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

- No nulls
- All numeric
- Preprocess Date Column

In [96]:
df.InvoiceDate.loc[:3]

0    2011-05-06 16:54:00
1    2011-05-11 07:35:00
2    2011-11-20 13:20:00
3    2011-11-22 12:07:00
Name: InvoiceDate, dtype: object

In [97]:
"""
preprocess date time col
"""

def preprocess_date_col(df, date_time_col: str):
    df[date_time_col] = pd.to_datetime(df[date_time_col], format='%Y-%m-%d %H:%M:%S')
    df['Year'] = df[date_time_col].dt.year
    df['WeekOfYear'] = df[date_time_col].dt.weekofyear
    df['Month'] = df[date_time_col].dt.month
    df['DayOfWeek'] = df[date_time_col].dt.dayofweek
    df['Weekend'] = (df[date_time_col].dt.weekday >= 5).astype(int)
    df['Hour'] = df[date_time_col].dt.hour
    
    df.drop([date_time_col], axis=1, inplace=True)
    return df

In [98]:
df = preprocess_date_col(df, 'InvoiceDate')

In [99]:
df.head(4)

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Year,WeekOfYear,Month,DayOfWeek,Weekend,Hour
0,6141,1583,144,3,3.75,14056.0,35,2011,18,5,4,0,16
1,6349,1300,3682,6,1.95,13098.0,35,2011,19,5,2,0,7
2,16783,2178,1939,4,5.95,15044.0,35,2011,46,11,6,1,13
3,16971,2115,2983,1,0.83,15525.0,35,2011,47,11,1,0,12


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    284780 non-null  int64  
 1   StockCode    284780 non-null  int64  
 2   Description  284780 non-null  int64  
 3   Quantity     284780 non-null  int64  
 4   UnitPrice    284780 non-null  float64
 5   CustomerID   284780 non-null  float64
 6   Country      284780 non-null  int64  
 7   Year         284780 non-null  int64  
 8   WeekOfYear   284780 non-null  int64  
 9   Month        284780 non-null  int64  
 10  DayOfWeek    284780 non-null  int64  
 11  Weekend      284780 non-null  int64  
 12  Hour         284780 non-null  int64  
dtypes: float64(2), int64(11)
memory usage: 28.2 MB


In [101]:
df.CustomerID = df.CustomerID.astype(int)

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    284780 non-null  int64  
 1   StockCode    284780 non-null  int64  
 2   Description  284780 non-null  int64  
 3   Quantity     284780 non-null  int64  
 4   UnitPrice    284780 non-null  float64
 5   CustomerID   284780 non-null  int64  
 6   Country      284780 non-null  int64  
 7   Year         284780 non-null  int64  
 8   WeekOfYear   284780 non-null  int64  
 9   Month        284780 non-null  int64  
 10  DayOfWeek    284780 non-null  int64  
 11  Weekend      284780 non-null  int64  
 12  Hour         284780 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 28.2 MB


**Reorder columns, rename and save to csv**

In [103]:
"""
reorder and rename columns
"""

TARGET_COL_NAME = 'UnitPrice'
ALL_COLS = list(df.columns)

# bring target col to end and create new col names
ALL_COLS.remove(TARGET_COL_NAME) # inplace
ALL_COLS = ALL_COLS + [TARGET_COL_NAME]
NEW_COL_NAMES = [f"f_{i}" if col != TARGET_COL_NAME else 'target' for i, col in enumerate(ALL_COLS)]

df = df[ALL_COLS]
df.columns = NEW_COL_NAMES

In [104]:
df.head(2)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,target
0,6141,1583,144,3,14056,35,2011,18,5,4,0,16,3.75
1,6349,1300,3682,6,13098,35,2011,19,5,2,0,7,1.95


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 13 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   f_0     284780 non-null  int64  
 1   f_1     284780 non-null  int64  
 2   f_2     284780 non-null  int64  
 3   f_3     284780 non-null  int64  
 4   f_4     284780 non-null  int64  
 5   f_5     284780 non-null  int64  
 6   f_6     284780 non-null  int64  
 7   f_7     284780 non-null  int64  
 8   f_8     284780 non-null  int64  
 9   f_9     284780 non-null  int64  
 10  f_10    284780 non-null  int64  
 11  f_11    284780 non-null  int64  
 12  target  284780 non-null  float64
dtypes: float64(1), int64(12)
memory usage: 28.2 MB


In [106]:
df.to_csv('../input/preprocessed_train_regression.csv')